In [3]:
# Import libraries
import pandas as pd
import requests
import random
import time
from datetime import datetime
# Parameters
GroupID = ['1846', '1789']
brands = ['HP']
# Define functions
def remove_single_category(df):
    category_counts = df.groupby('MasterCategoryID')['CategoryID'].nunique()
    single_category_mask = category_counts == 1
    df.loc[df['MasterCategoryID'].isin(single_category_mask[single_category_mask].index), ['CategoryID', 'CategoryName']] = [None, None]
    return df

def transform_category(row):
    if pd.isna(row['CategoryID']):
        master_category_id = row['MasterCategoryID']
        master_category_name = row['MasterCategoryName']
        category_id = row['MasterCategoryID']
        category_name = row['MasterCategoryName']
        is_category = 0
    else:
        master_category_id = row['MasterCategoryID']
        master_category_name = row['MasterCategoryName']
        category_id = row['CategoryID']
        category_name = row['CategoryName']
        is_category = 1

    if pd.isna(row['SubCategoryID']):
        sub_category_id = category_id
        sub_category_name = category_name
        is_sub_category = 0
    else:
        sub_category_id = row['SubCategoryID']
        sub_category_name = row['SubCategoryName']
        is_sub_category = 1

    return pd.Series([
        master_category_id, master_category_name, category_id, category_name,
        is_category, sub_category_id, sub_category_name, is_sub_category
    ])

def retrieve_product_ids(id):
    base_url = "https://tiki.vn/api/personalish/v1/blocks/listings"
    PARAMS = {"category": id, "page": 1}
    response = requests.get(base_url, headers=HEADERS, params=PARAMS)
    time.sleep(random.uniform(3.2, 8.7))
    data = response.json()
    total_page = data["paging"]["last_page"]

    product_data = []
    for page in range(1, total_page + 1):
        PARAMS = {"category": id, "page": page}
        response = requests.get(base_url, headers=HEADERS, params=PARAMS)
        time.sleep(random.uniform(3.2, 8.7))
        data = response.json()
        for item in data["data"]:
            product_id = item["id"]
            brand_name = item.get("brand_name", None)
            product_data.append({"product_id": product_id, "brand_name": brand_name})

    return product_data
# Set up header
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://tiki.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

In [15]:
# EXTRACT group of categories
URL = "https://api.tiki.vn/raiden/v2/menu-config?platform=desktop"

response = requests.get(URL, headers=HEADERS)
time.sleep(random.uniform(3.2, 8.7))

if response.status_code == 200:
    data = response.json()
    
    group_list = []
    group = data["menu_block"]["items"]
    for group in group:
      link = group["link"]
      group_id = link.split("/")[-1][1:]
      text = group["text"]

      if group_id in GroupID:
          group_list.append([group_id, text])
      
# group df    
group_df = pd.DataFrame(group_list, columns=["GroupID", "Name"])
# EXTRACT categories
category_list = []
for group_id, group_name in zip(group_df["GroupID"], group_df["Name"]):
    parent_url = f"https://tiki.vn/api/v2/categories?parent_id={group_id}"
    parent_response = requests.get(parent_url, headers=HEADERS)
    time.sleep(random.uniform(3.2, 8.7))
    if parent_response.status_code == 200:
        parent_data = parent_response.json()
        if not parent_data["data"]:
            category_list.append([group_id, group_name, None, None, None, None, None, None])
        else:
            for parent_category in parent_data["data"]:
                parent_id = parent_category["id"]
                parent_name = parent_category["name"]
                
                child_url = f"https://tiki.vn/api/v2/categories?parent_id={parent_id}"
                child_response = requests.get(child_url, headers=HEADERS)
                time.sleep(random.uniform(3.2, 8.7))
                
                if child_response.status_code == 200:
                    child_data = child_response.json()
                    if not child_data["data"]:
                        category_list.append([group_id, group_name, parent_id, parent_name, None, None, None, None])
                    else:
                        for child_category in child_data["data"]:
                            child_id = child_category["id"]
                            child_name = child_category["name"]
                            
                            type_url = f"https://tiki.vn/api/v2/categories?parent_id={child_id}"
                            type_response = requests.get(type_url, headers=HEADERS)
                            time.sleep(random.uniform(3.2, 8.7))
                            
                            if type_response.status_code == 200:
                                type_data = type_response.json()
                                if type_data["data"]:
                                    for type_item in type_data["data"]:
                                        type_id = type_item.get("id")
                                        type_name = type_item.get("name")
                                        category_list.append([group_id, group_name, parent_id, parent_name, child_id, child_name, type_id, type_name])
                                else:
                                    category_list.append([group_id, group_name, parent_id, parent_name, child_id, child_name, None, None])

category = pd.DataFrame(category_list, columns=["GroupID", "GroupName", "MasterCategoryID", "MasterCategoryName", "CategoryID", "CategoryName", "SubCategoryID", "SubCategoryName"])
cleaned_df = remove_single_category(category)
category[['MasterCategoryID', 'MasterCategoryName', 'CategoryID', 'CategoryName', 'isCategory', 'SubCategoryID', 'SubCategoryName', 'isSubCategory']] = category.apply(transform_category, axis=1, result_type='expand')
category["GroupID"] = category["GroupID"].astype(int)
category["MasterCategoryID"] = category["MasterCategoryID"].astype(int)
category["CategoryID"] = category["CategoryID"].astype(int)
category["SubCategoryID"] = category["SubCategoryID"].astype(int)

# master_category df
master_category_df = category[["MasterCategoryID", "GroupID", "MasterCategoryName"]].drop_duplicates()
master_category_df = master_category_df.rename(columns={"MasterCategoryName": "Name"})
# category df
category_df = category[["CategoryID", "MasterCategoryID", "CategoryName", "isCategory"]].drop_duplicates()
category_df = category_df.rename(columns={"CategoryName": "Name"})
# sub_category df
sub_category_df = category[["SubCategoryID", "CategoryID", "SubCategoryName", "isSubCategory"]].drop_duplicates()
sub_category_df = sub_category_df.rename(columns={"SubCategoryName": "Name"})

KeyboardInterrupt: 

In [ ]:
sub_category_df.to_csv('sub_category.csv', index=False, encoding='utf-8-sig')

In [17]:
master_category_df.dtypes

MasterCategoryID     int32
GroupID              int32
Name                object
dtype: object

In [11]:
category_df.dtypes

CategoryID           int32
MasterCategoryID     int32
Name                object
isCategory           int64
dtype: object

In [12]:
sub_category_df.dtypes

SubCategoryID     int32
CategoryID        int32
Name             object
isSubCategory     int64
dtype: object

In [30]:
import pyodbc
import json
import pandas as pd

# Configuration for the SQL Server connection
server = 'flexiboard.database.windows.net'
database = 'DevDB'
username = 'flexiboard-admin'
password = 'Final@Project2024'
driver = 'ODBC Driver 18 for SQL Server'

# Set the desired timeout value in seconds
timeout = 60

# Connection string
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Timeout={timeout};"
)

conn = pyodbc.connect(conn_str)
print("Connected to Azure SQL Server")

# Create a cursor
cursor = conn.cursor()

# Drop tables in the correct order
try:
    # Drop the view that references the tables
    cursor.execute("DROP VIEW IF EXISTS SalesLT.vProductAndDescription")

    # Get table information
    tables = []
    cursor.execute("SELECT DISTINCT KCU.TABLE_NAME, KCU.CONSTRAINT_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE KCU INNER JOIN INFORMATION_SCHEMA.TABLE_CONSTRAINTS TC ON KCU.CONSTRAINT_NAME = TC.CONSTRAINT_NAME WHERE TC.CONSTRAINT_TYPE = 'FOREIGN KEY' AND KCU.TABLE_SCHEMA = 'SalesLT'")
    for table_name, constraint_name in cursor.fetchall():
        tables.append((table_name, constraint_name))

    # Sort tables based on foreign key constraints
    sorted_tables = sorted(set(tables), key=lambda x: tables.count(x), reverse=True)

    # Drop tables referencing SalesLT.ProductCategory first
    product_category_refs = []
    cursor.execute("SELECT DISTINCT OBJECT_NAME(PARENT_OBJECT_ID) FROM SYS.FOREIGN_KEYS WHERE REFERENCED_OBJECT_ID = OBJECT_ID('SalesLT.ProductCategory')")
    for ref_table in cursor.fetchall():
        product_category_refs.append(ref_table[0])

    for ref_table in product_category_refs:
        cursor.execute(f"DROP TABLE SalesLT.{ref_table}")
        print(f"Dropped table: SalesLT.{ref_table}")

    # Drop remaining tables in the correct order
    for table_name, _ in sorted_tables:
        if table_name not in product_category_refs:
            cursor.execute(f"DROP TABLE SalesLT.{table_name}")
            print(f"Dropped table: SalesLT.{table_name}")

    conn.commit()
    print("Tables dropped successfully!")
except pyodbc.Error as e:
    print(f"Error dropping tables: {e}")
    conn.rollback()

Connected to Azure SQL Server
Dropped table: SalesLT.Product
Error dropping tables: ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Cannot DROP TABLE 'SalesLT.ProductCategory' because it is being referenced by object 'vGetAllCategories'. (3729) (SQLExecDirectW)")
